In [49]:
# https://pub.towardsai.net/step-by-step-guide-in-creating-your-own-emotion-recognition-system-b8aba98134c8
# https://github.com/priya-dwivedi/face_and_emotion_detection/blob/master/src/EmotionDetector_v2.ipynb

In [50]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import regularizers
from keras.regularizers import l1

In [51]:
import face_recognition
from pathlib import Path

In [52]:
import numpy as np
import pandas as pd
import os
import cv2
import glob
from PIL import Image

In [53]:
import matplotlib.pyplot as plt

In [54]:
model = load_model("./emotion_detector_models/model.hdf5")
predicted_class = np.argmax(model.predict(face_image))

OSError: SavedModel file does not exist at: ./emotion_detector_models/model.hdf5/{saved_model.pbtxt|saved_model.pb}

In [55]:
# keeping personal directory info safe

folder_inp1 = Path.cwd().parent.parent/'inp1'

all_file = 'Fer.csv'

train_dir_name = 'train'
train_file_pth = folder_inp1/train_dir_name
train_file_name = 'Training_Data.csv'
train_csv = train_file_pth/train_file_name

testing_dir_name = 'test_validation'
testing_file_pth = folder_inp1/testing_dir_name
testing_file_name = 'Testing_Data.csv'
testing_csv = testing_file_pth/testing_file_name

validation_dir_name = 'test_validation'
validation_file_pth = folder_inp1/validation_dir_name
validation_file_name = 'Validation_Data.csv'
validation_csv = validation_file_pth/validation_file_name

In [56]:
train_df = pd.read_csv(f"{train_csv}", delimiter=",")

In [57]:
train_df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [58]:
num_classes = 7
img_rows, img_cols = 48, 48
batch_size = 512

In [59]:
train_data_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest')

In [60]:
train_generator = train_data_generator.flow_from_directory(
    train_file_pth,
    target_size=(48,48),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode='categorical')


Found 56706 images belonging to 3 classes.


In [61]:
def extract_features(filename):
    
    """
    separates features from targets
    
    """
    X = []
    Y = []
    headers = True
    for line in open(filename):
        if headers:
          headers = False
        else:
            row = line.split(',')
            Y.append(int(row[0]))
            X.append([int(feature) for feature in row[1].split()])

    X,Y = np.array(X)/255.0, np.array(Y)

    return (X,Y)

In [62]:
train_X, train_Y = extract_features(train_csv)
vald_X, vald_Y = extract_features(validation_csv)
test_X, test_Y = extract_features(testing_csv)

In [63]:
train_X = np.array(train_X,'float32')
train_Y = np.array(train_Y,'float32')
vald_X = np.array(vald_X,'float32')
vald_Y = np.array(vald_Y,'float32')
test_X = np.array(test_X,'float32')
test_Y = np.array(test_Y,'float32')

In [64]:
def plot(history):
    
    """
    analyzes model performance as the number of epochs increases
    
    """
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Acc')
    plt.legend()
    plt.figure()
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [65]:
def emotion_analysis(emotions):
    
    """
    Analyzes predicted emotion by constructing a barplot.
    Shows confidence level of model when predicting emotion.
    
    """
    
    objects = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    y_pos = np.arange(len(objects))
    plt.bar(y_pos, emotions, align='center', alpha=0.9)
    plt.tick_params(axis='x', which='both', pad=10,width=4,length=10)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')
    
    plt.show()

In [66]:
from skimage import io
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

def display_predictions(img_file,model):
    
    """
    Predicts from own image.
    
    """
    img = image.load_img(img_file, grayscale=True, target_size=(48, 48))
    show_img=image.load_img(img_file, grayscale=False, target_size=(200, 200))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)

    x /= 255

    custom = model.predict(x)
    #print(custom[0])
    emotion_analysis(custom[0])

    x = np.array(x, 'float32')
    x = x.reshape([48, 48]);

    plt.gray()
    plt.imshow(show_img)
    plt.show()

    m=0.000000000000000000001
    a=custom[0]
    
    for i in range(0,len(a)):
        if a[i]>m:
            m=a[i]
            ind=i
          
    print('Expression Prediction:',objects[ind])

In [67]:
N, D = train_X.shape
vald_n,vald_d = vald_X.shape
test_n, test_d = test_X.shape
train_X = train_X.reshape(N, 48, 48, 1)
vald_X = vald_X.reshape(vald_n, 48, 48, 1)
test_X = test_X.reshape(test_n,48,48,1)

In [68]:
def base_model():
#1st convolution layer
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
    model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
    model.add(Dropout(0.5))
    #2nd convolution layer
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
    model.add(Dropout(0.5))
    #3rd convolution layer
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
    model.add(Flatten())
    #fully connected neural networks
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_labels, activation='softmax'))
    return model

In [69]:
def tuned_model():
    model = Sequential()
    input_shape = (48,48,1)
    #1st convolution layer
    model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    #2nd convolution layer
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    #3rd convolution layer
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7))
    model.add(Activation('softmax'))

    my_optimiser = tf.keras.optimizers.Cris(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Cris')
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=my_optimiser)
    
    return model

In [70]:
history = model_1.fit(train_X,     
            Y_train, 
            batch_size=64, 
            epochs=80, 
            verbose=1, 
            validation_data=(val_X,Y_val),
            shuffle=True,
            )

NameError: name 'model_1' is not defined

In [ ]:
plot(history) # This will display the accuracy and loss graphs.